In [5]:
%load_ext autoreload
%autoreload 2

from semantic_parser.modules.verdant.actions import *
from semantic_parser.modules.verdant.openai_utils import *
from semantic_parser.decompose_agent import DecomposeAgent
from semantic_parser.action_protocol import ActionRegistry, ActionOutput
from semantic_parser.llm_client import AzureOpenAIClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os

client = OpenAIUtils(api_key=os.getenv("AZURE_OPENAI_API_KEY"))
db_manager = DatabaseManager(
    visible_tables=[
        "FundCashFlow",
        "FundInvestmentProperties",
        "FundInvestmentTimeProperties",
        "FundTimeProperties",
        "HandTransformedPortfolioInvestmentswithCFRaw"
    ]
)

# Create the action registry
action_registry = ActionRegistry()

# Register built-in and example actions
action_registry.register(FetchRelatedColumns(openai_client=client))
action_registry.register(FetchBackgroundKnowledge(openai_client=client))
action_registry.register(SketchSQL(openai_client=client))
# action_registry.register(GenerateSQL(openai_client=client))
# action_registry.register(ExecuteSQL(db_manager=db_manager))
action_registry.register(GenerateAndExecuteSQL(openai_client=client, db_manager=db_manager))
action_registry.register(Finish(openai_client=client))  # Updated to include openai_client

llm_client = AzureOpenAIClient(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint="https://ovalnairr.openai.azure.com/",
    api_version="2024-12-01-preview",
)
decompose_agent = DecomposeAgent(
    llm_client=llm_client,
    action_registry=action_registry
)


In [14]:
query = "On a year-on-year basis (4Q24 vs 4Q23), which funds saw an uplift to overall MOIC (i.e., Fund MOIC) in 2024?"
result = await decompose_agent._judge_complexity(query, context=None)
print(result)

TaskComplexity(is_simple=False, reasoning='The task requires identifying the correct table(s) and columns containing quarterly fund-level MOIC data, then comparing 4Q24 to 4Q23 and filtering funds with an increase. This involves at least two actions (discovering relevant columns and then querying/comparing), so it cannot be accomplished with a single action call.', recommended_action=None, recommended_parameters=None)


In [22]:
result = await decompose_agent._decompose_task(query, context=None)

In [23]:
subtasks = result.subtasks
composition_operators = result.composition_operators
control_flow = result.control_flow
reasoning = result.reasoning


In [24]:
for i, subtask in enumerate(subtasks):
    print(f"Subtask {i+1}: {subtask}")

Subtask 1: SubTask(subtask_id='subtask_1', description='Fetch the table and column information required to identify where Fund MOIC by quarter is stored (e.g., fund identifier, quarter end date, MOIC value).', depends_on=[])
Subtask 2: SubTask(subtask_id='subtask_2', description='Using the schema knowledge from subtask_1, run a SQL query that pulls each fund’s MOIC for 4Q24 and 4Q23, compares the two, and returns the funds whose MOIC increased (4Q24 MOIC > 4Q23 MOIC).', depends_on=[])
